In [30]:
import pandas as pd 
import json
import numpy as np
import plotly.graph_objects as go
import plotly.graph_objects as go
import numpy as np

import sys
sys.path.append('..')
import ENVIRONMENT_VARIABLES as EV

In [31]:
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round"]
year =2024

In [32]:
# Initialize an empty dictionary to store data for each year
years_arr = {}

# Open the JSON file containing NRL data
with open(f'../data/nrl_data_{year}.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    
    # Extract NRL data from the loaded JSON
    data = data['NRL']

    years_arr[year] = data[0][str(year)]


In [33]:
# Create a DataFrame with columns representing combinations of team and variable names
df = pd.DataFrame(columns=[f"{team} {variable}" for team in EV.TEAMS for variable in variables])

In [ ]:
# Initialize an empty list to store data for all rounds
all_store = []

# Iterate over each round (assuming 26 rounds)
for round in range(0, 26):
    try:
        # Extract data for the current round
        round_data = years_arr[year][round][str(round+1)]
        
        # Create an empty feature array 
        round_store = np.zeros([len(EV.TEAMS)*len(variables)], dtype=int)
        round_teams = []
        
        # Iterate over each game in the round data
        for game in round_data:
            # Extract information about the game
            h_team = game['Home']
            h_score = int(game['Home_Score'])
            a_team = game['Away']
            a_score = int(game['Away_Score'])
            
            # Determine win or lose for each team
            h_team_win = h_score >= a_score
            a_team_win = a_score >= h_score
            
            # Determine home team status
            h_home = 1
            a_home = 0
            
            # Determine versus index
            h_versus = EV.TEAMS.index(a_team)
            a_versus = EV.TEAMS.index(h_team)
            
            # Determine defense (points let in)
            h_team_defense = a_score
            a_team_defense = h_score  
            
            # Determine attack points scored
            h_team_attack = h_score 
            a_team_attack = a_score   
            
            # Determine margin
            h_team_margin =  h_score - a_score   
            a_team_margin =  a_score - h_score        
            
            # Keep track of which teams played to work out which teams had a bye 
            round_teams.append(h_team)
            round_teams.append(a_team)
            
            # Find the index of the team in the overarching array 
            a_team_idx = EV.TEAMS.index(a_team)
            h_team_idx = EV.TEAMS.index(h_team)
            
            # Determine feature map index
            a_team_idx_fm = a_team_idx * len(variables)
            h_team_idx_fm = h_team_idx * len(variables)
            
            # Populate the data for away team
            round_store[a_team_idx_fm] = year
            round_store[a_team_idx_fm+1] = a_team_win
            round_store[a_team_idx_fm+2] = a_team_defense
            round_store[a_team_idx_fm+3] = a_team_attack
            round_store[a_team_idx_fm+4] = a_team_margin
            round_store[a_team_idx_fm+5] = a_home
            round_store[a_team_idx_fm+6] = a_versus
            round_store[a_team_idx_fm+7] = round+1
            
            # Populate the data for home team
            round_store[h_team_idx_fm] = year
            round_store[h_team_idx_fm+1] = h_team_win
            round_store[h_team_idx_fm+2] = h_team_defense
            round_store[h_team_idx_fm+3] = h_team_attack
            round_store[h_team_idx_fm+4] = h_team_margin
            round_store[h_team_idx_fm+5] = h_home
            round_store[h_team_idx_fm+6] = h_versus
            round_store[h_team_idx_fm+7] = round+1
            
        # Determine teams with a bye
        bye_teams = list(set(EV.TEAMS) - set(round_teams))
        
        # Assign values for teams with a bye
        for bye_team in bye_teams:
            b_team_idx = teams.index(bye_team)
            b_team_idx_fm = b_team_idx * len(variables)
            round_store[b_team_idx_fm] = year
            round_store[b_team_idx_fm+1] = 0
            round_store[b_team_idx_fm+2] = 0
            round_store[b_team_idx_fm+3] = 0
            round_store[b_team_idx_fm+4] = 0
            round_store[b_team_idx_fm+5] = 0
            round_store[b_team_idx_fm+6] = 0
            round_store[b_team_idx_fm+7] = round+1
            
        # Append the round data to the all_store list
        all_store.append(round_store)
        
        # Add the new row to the DataFrame using loc
        df.loc[len(df)] = round_store
    except:
        pass

In [35]:
attack_columns = [col for col in df.columns if 'Attack' in col]
defense_columns = [col for col in df.columns if 'Defense' in col]
df_attack = df[attack_columns].T
df_attack = df_attack.rename(columns={0: 'Attack'})

df_defense = df[defense_columns].T
df_defense = df_defense.rename(columns={0: 'Defense'})


df_attack['Teams'] = teams 
df_defense['Teams'] = teams 


df_attack.reset_index(drop=True, inplace=True)
df_defense.reset_index(drop=True, inplace=True)


merged_df = pd.merge(df_attack, df_defense, on='Teams')

In [36]:


# Create a bar plot using Plotly
fig = go.Figure(data=[
    go.Bar(
        x=df_attack['Teams'], 
        y=df_attack['Attack'],
        marker_color=[EV.TEAM_COLOURS[team] for team in df_attack['Teams']],
        name='Attack'
    ),
    go.Bar(
        x=merged_df['Teams'], 
        y=-merged_df['Defense'],  # Negative values for defense
        name='Defense',
        marker_color=[EV.TEAM_COLOURS_INVERSE[team] for team in merged_df['Teams']]
    ),
])

# Customize layout
fig.update_layout(
    title='Team Performance',
    xaxis_title='Team',
    yaxis_title='Attack and Defense',
    plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot background
    paper_bgcolor='rgba(0, 0, 0, 0)',  # Transparent paper background
    font=dict(color='white'),  # White font color
     barmode='relative', 
    template='plotly_dark'  # Dark theme template
)

# Show plot
fig.show()
